## Introduction

Welcome to our final project! The goal of this project is to construct a __Car Buyer's Guide for Master students__.

We used Python for programming and R for statistic summary and visualization. Thus, to run our project, make sure the Anaconda and R are installed and we need Rpy2 package to run R code:

    $ conda install -c r rpy2

Next, we will describe the problem scope and the key assumptions of our guide.

### Dataset Scope

We chose 9 well-known car manufacturers over the world, and for each manufacturer, we chose 2 representative car models for our analysis (e.g. Honda Accord). These are classic car models that have been proved by time and the total sales (each of the models returns at least 1000 records per year when we searched at Cars.com, which reflects their popularity). Our dataset includes the detailed information of these 18 models from 2014 to 2018, and all of our data is scraped from Cars.com. We have a total of 269,353 car records. For each car model, we have more than one trims (different sub-categories e.g. Honda Accord EX, Honda Accord Sport). To filter our less popular trims in our dataset, we discarded trims that have less than 100 records per year. As a result, we have 24 modelTrims and 118,258 records. This is the dataset we ended up using in our analysis.

### Key Assumptions

We have the following assumptions for our projects:
1. Master students have no bias on new cars and second-hand cars, they care about the price and availability.
2. The main focus of the Master students is to buy a car that maintains its value well so they can sell it when they graduate from the University.
3. We only keep track of the car records in the recent 5 years because we think Master students are not likely to buy cars that do not have car insurance/warranty and need frequent maintenance.

## Outline

- [Data Collection and Processing](#Data-Collection-and-Processing)

We then proposed two questions to this dataset, and constructed two mini-labs to seek for answers to the questions:
- [Should students buy new cars or used cars with few millages?](#Lab1-Compare-new-cars-with-used-2018-cars)
- [How the price changes in used cars from 2014 to 2018?](#Lab2-Model-the-price-change-in-used-cars-from-2014-to-2018)

Finally, we have:
- [Future Work](#Future-Work)
- [Summary and References](#Summary-and-References)

## Data Collection and Processing

Below is the list of 18 car models we initially used for web scraping.

#### American
- Ford Focus
- Ford F-150
- Chevrolet Cruze
- Chevrolet Malibu

#### German
- Audi A4
- Audi A6
- Mercedes-Benz C
- Mercedes-Benz E
- Volkswagen Passat
- Volkswagen Jetta

#### Asian
- Honda Civic
- Honda Accord
- Toyota Prius
- Toyota Camry
- Kia Optima
- Kia Sorento
- Hyundai Elantra
- Hyundai Sonata

### Collection

We scraped all statistics for the 18 models from 2014 to 2018 from Cars.com.

We download one web page every 2.5 seconds. One search returns 50 pages and each page returns the records of 100 cars. Thus, each model returns a maximum of 5000 records per year.

In [1]:
import io, time, json
import requests
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def retrieve_html(url):
    html = urllib.request.urlopen(url).read()
    return html

def remove_comma(string):
    return string.replace(',', '')

#calculate how many pages should download for one search
def pages(number):
    if number % 100 != 0:
        return int(number / 100 + 1)
    else:
        return number/ 100

#generate the url for search.
def gen_url(mkId, mdId, yrId, page=1):
    return "https://www.cars.com/for-sale/searchresults.action/?{}&{}&page={}&perPage=100&rd=99999&searchSource=GN_REFINEMENT&showMore=true&sort=relevance&{}&zc=15213".format(mdId, mkId, page,yrId)

In [3]:
brand_and_model = {("Honda", "Accord"): ("mkId=20017","mdId=20606"),
("Honda", "Civic"): ("mkId=20017","mdId=20823"),
("Toyota","Camry"): ("mkId=20088", "mdId=20800"),
("Toyota","Prius"): ("mkId=20088", "mdId=21751"),
("Audi","A4"): ("mkId=20049","mdId=20596"),
("Audi","A6"): ("mkId=20049","mdId=20598"),
("Mercedes-Benz","C"): ("mkId=20028", "mdId=30022818,36279856,36279857,20745"),
("Mercedes-Benz","E"): ("mkId=20028", "mdId=30019351,36290282,36322313,21014"),
("Ford","Focus"): ("mkId=20015", "mdId=21138"),
("Ford","F-150"): ("mkId=20015", "mdId=21095"),
("Chevrolet","Malibu"): ("mkId=20053", "mdId=21413"),
("Chevrolet","Cruze"):  ("mkId=20053", "mdId=35026"),
("Kia","Sorento"): ("mkId=20068", "mdId=22084"),
("Kia","Optima"): ("mkId=20068", "mdId=21740"),
("Hyundai","Elantra"): ("mkId=20064", "mdId=21053"),
("Hyundai","Sonata"): ("mkId=20064", "mdId=22146"),
("Volkswagen","Passat"): ("mkId=20089", "mdId=21758"),
("Volkswagen","Jetta"): ("mkId=20089", "mdId=21285")
}
year_list = [("2018","yrId=35797618"), ("2017","yrId=30031936"), ("2016","yrId=58487"), ("2015","yrId=56007"), ("2014","yrId=51683")]
years = ["2018", "2017", "2016", "2015", "2014"]

In [4]:
def retrieve():
    p = 0
    for (brand, model), (mkId, mdId) in brand_and_model.items():
        for (year, yrId) in year_list:
            start_time = time.time()
            curr_url = gen_url(mkId, mdId, yrId)
            print(curr_url)
            html = retrieve_html(curr_url)
            root = BeautifulSoup(html,"html.parser")
            matchcount = root.find("div", {'class':'matchcount'}, recursive=True)
            count = matchcount.find("span", {'class':'count'}, recursive=True)
            count = int(remove_comma(count.contents[0]))
            print(count)
            number_pages = min(pages(count),50)
            print(number_pages)

            ans = []
            for i in range(1, number_pages+1):
                curr_url = gen_url(mkId, mdId, yrId, i)
                html = retrieve_html(curr_url)
                root = BeautifulSoup(html,"html.parser")
                root = root.find("script")
                content = root.contents[0] 
                start = content.find("CARS.digitalData") + len("CARS.digitalData = ")
                end = content.rfind(";")
                print(i, start, end)
                vehicle_list = json.loads(content[start:end]).get("page").get("vehicle")
                ans += vehicle_list
                end_time = time.time()
                print("time: {}".format(end_time-start_time))
            print(p,"#"*99)
            p += 1
            np.save("data/{}|{}|{}.npy".format(brand, model, year), ans)

### Processing

As we mentioned in the dataset scope, we discarded trims that have less than 100 records per year.

In [5]:
def take_trim(l):
    return [i["trim"] for i in l]

In [6]:
def all_greater(l, n):
    for _, e in l:
        if e < n:
            return False
    return True

In [7]:
brand_and_model_and_trim = []
sum_data = 0
raw_sum_data = 0

database_list = []
for (brand, model), _ in brand_and_model.items():
    a = []
    for year in years:
        name = "data/{}|{}|{}.npy".format(brand, model, year)
        raw_sum_data += len(np.load(name))
        a.append(np.load(name))

    trim = [Counter(take_trim(i)) for i in a]
    car_name = [name for name, number in trim[-1].items()]
    for name in car_name:
        ans = [(name, i[name]) for i in trim]
        if all_greater(ans, 100):
            for i in a:
                database_list += list(filter(lambda x: x['trim'] == name, i))
                
            brand_and_model_and_trim.append((brand, model, name))
            
            num = sum([num for name, num in ans])
            sum_data += num
            
print('the size of raw dataset: ' ,raw_sum_data)
print('the size of dataset after discarding trims less than 100 records per year: ',sum_data)
print('the number of modelTrim', len(brand_and_model_and_trim))
for i in brand_and_model_and_trim:
    print(i)

the size of raw dataset:  269353
the size of dataset after discarding trims less than 100 records per year:  123319
the number of modelTrim 24
('Honda', 'Accord', 'EX')
('Honda', 'Accord', 'Sport')
('Honda', 'Civic', 'EX')
('Honda', 'Civic', 'LX')
('Toyota', 'Camry', 'XLE')
('Toyota', 'Camry', 'SE')
('Toyota', 'Camry', 'LE')
('Toyota', 'Prius', 'Two')
('Audi', 'A4', '2.0T Premium quattro')
('Mercedes-Benz', 'C', 'C 300')
('Ford', 'Focus', 'SE')
('Ford', 'Focus', 'Titanium')
('Ford', 'Focus', 'S')
('Ford', 'F-150', 'XLT')
('Ford', 'F-150', 'XL')
('Ford', 'F-150', 'Lariat')
('Chevrolet', 'Malibu', '1LT')
('Kia', 'Sorento', 'EX')
('Kia', 'Sorento', 'LX')
('Kia', 'Optima', 'EX')
('Kia', 'Optima', 'LX')
('Hyundai', 'Elantra', 'SE')
('Hyundai', 'Sonata', 'SE')
('Hyundai', 'Sonata', 'Limited')


We put database_list into the dataframe, then we discard records with missing values. We only maintain the 7 columns that are useful for our analysis. They are: __make mileage model price stockType trim year__

In [8]:
database = pd.DataFrame(database_list)
database = database.drop(['certified','bodyStyle', 'customerId', 'detail', 'listingId', 'makeId', 'modelId', 'priceBadge', 'privateSeller', 'type', 'vin'],axis=1)
database = database.dropna(axis=0, how='any')
database = database.reset_index(drop=True)
database.head(5)
#118268 rows × 7 columns

,make,mileage,model,price,stockType,trim,year
0,Honda,1242.0,Accord,23999.0,Used,EX,2018
1,Honda,13.0,Accord,28360.0,New,EX,2018
2,Honda,6.0,Accord,27935.0,New,EX,2018
3,Honda,95.0,Accord,24999.0,Used,EX,2018
4,Honda,0.0,Accord,28360.0,New,EX,2018


Now we can calculate the statistic characteristics of our dataset: Mean, Standard Deviation, Max, and Min.

In [9]:
pd.set_option('display.max_rows', 200)
database.groupby(['make', 'model', 'trim', 'year'])['price'].agg({'Mean':np.mean, 'SD':np.std, 'Max':max, 'Min':min})


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


Mean            SD  \
make          model   trim                 year                               
Audi          A4      2.0T Premium quattro 2014  19589.502994   2500.716786   
                                           2015  21716.237209   2500.261671   
                                           2016  24397.427184   2710.466596   
                                           2017  32328.988235   4891.873006   
                                           2018  44829.784375   3130.406618   
Chevrolet     Malibu  1LT                  2014  13027.935310   1904.132201   
                                           2015  14786.434127   1588.342803   
                                           2016  16885.434599   2238.801933   
                                           2017  17843.976954   3036.857499   
                                           2018  23108.229469   3330.062325   
Ford          F-150   Lariat               2014  30266.178431   3935.974535   
                                           2015  36462.223114   3869.889542   
                                           2016  40259.255841   6112.979862   
                                           2017  45098.096519   8707.446403   
                                           2018  51835.044944  15139.875038   
                      XL                   2014  21503.876344   5923.347406   
                                           2015  27313.953431   4331.744686   
                                           2016  27127.603659   4503.044615   
                                           2017  31559.668950   7311.222590   
                                           2018  34343.778423   5857.272876   
                      XLT                  2014  25893.233497   3678.043555   
                                           2015  29746.441536   3350.014666   
                                           2016  31964.552098   3734.186572   
                                           2017  32858.792395   5559.411235   
                                           2018  36930.338583   5368.264419   
              Focus   S                    2014   9601.186916   1671.163261   
                                           2015  11202.232323   2633.809559   
                                           2016  11968.078947   1866.531662   
                                           2017  14339.398058   3688.790086   
                                           2018  15996.699917   2348.369767   
                      SE                   2014  10480.578947   1666.328955   
                                           2015  12112.682548   1748.544463   
                                           2016  12698.078924   1794.537743   
                                           2017  15752.745057   3385.505085   
                                           2018  17744.976808   2628.950764   
                      Titanium             2014  12472.460123   1842.587842   
                                           2015  14884.068627   2106.079173   
                                           2016  16776.850340   4098.605388   
                                           2017  20917.081761   3386.378758   
                                           2018  22692.509202   2911.369014   
Honda         Accord  EX                   2014  15324.655914   1991.783096   
                                           2015  17226.717125   1496.427090   
                                           2016  20255.445993   2293.024661   
                                           2017  22927.255102   2854.371042   
                                           2018  27939.628272    872.223726   
                      Sport                2014  15195.505128   1995.528487   
                                           2015  17183.755454   1552.314119   
                                           2016  20055.170868   1849.473371   
                                           2017  22362.941548   1927.009134   
                            

You may notice that we have F-150 Lariat in 2018 with the max price of 131,695 dollars when the average is 51,835 dollars. It is obvious that this record is not ideal to recommend to students. Same applies to Kia Sorento LX 2017 where it has the max price of 224,253.
Now it is time to reduce these outlines.

### Reduce Outliers

We equally cut the max price to 10 ranges for each modelTrim and we count the number of cars that fall into each price range.

In [10]:
quartiles = pd.cut(database.price, 10)
database.price.groupby(quartiles).count()
#database[database.make == 'Ford'][database.model == 'F-150'][database.trim == 'Lariat']

price
(-224.253, 22425.3]     75184
(22425.3, 44850.6]      40443
(44850.6, 67275.9]       2589
(67275.9, 89701.2]         42
(89701.2, 112126.5]         7
(112126.5, 134551.8]        2
(134551.8, 156977.1]        0
(156977.1, 179402.4]        0
(179402.4, 201827.7]        0
(201827.7, 224253.0]        1
Name: price, dtype: int64

We can see there are 10 expensive cars with price over 89701 dollars. Now let's take a look at these outliers.

In [11]:
database[database.price > 89701.2]

,make,mileage,model,price,stockType,trim,year
74583,Ford,11.0,F-150,131695.0,New,Lariat,2018
74622,Ford,30.0,F-150,109195.0,New,Lariat,2018
74638,Ford,5.0,F-150,110015.0,New,Lariat,2018
74674,Ford,12.0,F-150,104945.0,New,Lariat,2018
74698,Ford,12.0,F-150,104945.0,New,Lariat,2018
74702,Ford,5.0,F-150,105999.0,New,Lariat,2018
74705,Ford,12.0,F-150,104945.0,New,Lariat,2018
74721,Ford,3.0,F-150,116695.0,New,Lariat,2018
76057,Ford,8.0,F-150,98000.0,New,Lariat,2016
90992,Kia,39267.0,Sorento,224253.0,Used,LX,2017


You may notice the Ford Lariat and Kia LX with very high price. We decided to discard these outliers from our dataset and move on.

In [12]:
database = database[database.price < 89701.2]
# save data to csv for future use
database.to_csv("car_data.csv")
#118258 rows × 7 columns

### Lab1 Compare new cars with used 2018 cars

This lab intends to address the question that if the Master student should buy a new car or a used 2018 cars. To minimize the influence of depreciation, we only choose used cars in 2018.

Two criterions are considered:

(1) The availability of cars

(2) The price of cars

Let's check the number of new cars and used 2018 cars. For new cars, we extract cars with stockType 'New'.

In [13]:
print("# Used cars: " , len(database[database.year == 2018][database.stockType == 'Used']))

# Used cars:  2487


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [14]:
print('# New cars: ' , len(database[database.stockType == 'New']))

# New cars:  33107


The number of new car is 13.3 times as many as used 2018 cars. Thus, new cars has better availability.

Next, we check the mean price of each modelTrim of two distributions. p1 is the mean price of used 2018 cars.

In [15]:
p1 = database[database.year == 2018][database.stockType == 'Used'].groupby(['make', 'model', 'trim'])['price'].mean()
print(p1)
p1 = p1.tolist()

make           model    trim                
Audi           A4       2.0T Premium quattro    38273.058824
Chevrolet      Malibu   1LT                     20256.095238
Ford           F-150    Lariat                  44678.072917
                        XL                      32756.800000
                        XLT                     35438.973881
               Focus    S                       15929.333333
                        SE                      15976.977778
                        Titanium                20291.500000
Honda          Accord   EX                      25194.833333
                        Sport                   26523.454545
               Civic    EX                      21367.473684
                        LX                      19262.161290
Hyundai        Elantra  SE                      15428.466667
               Sonata   Limited                 25763.866667
                        SE                      18195.054054
Kia            Optima   EX              

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


p2 is the mean price of new cars.

In [16]:
p2 = database[database.stockType == 'New'].groupby(['make', 'model', 'trim'])['price'].mean()
print(p2)
p2 = p2.tolist()

make           model    trim                
Audi           A4       2.0T Premium quattro    45194.246835
Chevrolet      Malibu   1LT                     23579.311813
Ford           F-150    Lariat                  55159.642570
                        XL                      34401.214196
                        XLT                     40795.029364
               Focus    S                       16028.426699
                        SE                      17930.038996
                        Titanium                22193.916786
Honda          Accord   EX                      27911.782500
                        Sport                   26203.874419
               Civic    EX                      21988.130694
                        LX                      20369.041801
Hyundai        Elantra  SE                      17559.787213
               Sonata   Limited                 27948.261841
                        SE                      22043.130293
Kia            Optima   EX              

By comparing two dataframes, except one modelTrims (Honda, Accord, Sport), the new price of other 23 modelTrims are greater than 2018 used prices.

In [17]:
drop_list = []
for i in range(len(p1)):
    drop_list.append((p2[i] - p1[i])/p2[i])
# (new cars - used cars) / new cars
drop_list = np.array(drop_list)
drop_list.mean()

0.10403215095985517

### Result of lab1:

As a result, the used car is at least 10.4% cheaper than the new cars, while the availability of the new cars is better than the used. Hence, it is a trade-off problem that whether a student should spend more time on searching for used cars or spend more money on buying new cars.

## Lab2 Model the price change in used cars from 2014 to 2018

### Statistical Analysis

To address the second question, a linear mixed model is used to determine what are the significant predictors of used car price. Linear mixed effects model is the extension of the linear model, so it is strongly related to the techniques presented in the course.

Linear mixed modeling is useful for this analysis because the dependent variable has a nested hierarchical structure within car trim (level). The coefficients of predictor variables were estimated using maximum likelihood estimation (MLEs) since we want to compare models that differ in fixed effects terms. The significance of predictor variables was depicted by their t-values. All t ratios with absolute values less than 2.8 are significant at p< 0.01. Used cars with mileage over 100,000 miles (0.97%) were considered outliers and therefore excluded from the models. 

We will use R for this lab since it has easy-to-use statistical tools. We first load our dataset from csv file.

In [18]:
%load_ext rpy2.ipython
%R carsZW <- read.csv('car_data.csv', sep = ",", header = T)
%R head(carsZW)

,X,make,mileage,model,price,stockType,trim,year
1,0,Honda,1242.0,Accord,23999.0,Used,EX,2018
2,1,Honda,13.0,Accord,28360.0,New,EX,2018
3,2,Honda,6.0,Accord,27935.0,New,EX,2018
4,3,Honda,95.0,Accord,24999.0,Used,EX,2018
5,4,Honda,0.0,Accord,28360.0,New,EX,2018
6,5,Honda,0.0,Accord,26136.0,New,EX,2018


In [19]:
# In this lab we only analyze used cars
%R usedcars<-subset(carsZW, stockType=="Used")
%R summary(usedcars$price)

array([     0.       ,  13887.       ,  16587.       ,  19069.1590821,
        21999.       ,  83000.       ])

Since Python has no popular linear mixed effects model package, we use lme4 package in R.

Although we can use Rpy2 package to run R code in jupyter notebook, the outputs are lack of description it should have.

For example, the output of summary function in R should be like as the cell below, but the jupyter notebook only return the array as above.

In such cases, we complete the description in the next cell.

In [20]:
#    Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
#    2995   13887   16587   19070   21999   83000

In [21]:
# We call the tuple of (make, model, trim) as modelTrim.
%R usedcars$modelTrim<-paste(usedcars$make, usedcars$model, usedcars$trim)

array(['Honda Accord EX', 'Honda Accord EX', 'Honda Accord EX', ...,
       'Hyundai Sonata Limited', 'Hyundai Sonata Limited',
       'Hyundai Sonata Limited'],
      dtype='<U28')

Then we load lme4 package which fits linear and generalized linear mixed-effects models.

In [22]:
%R library(lme4)

/usr/local/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: Matrix

  warnings.warn(x, RRuntimeWarning)


array(['lme4', 'Matrix', 'tools', 'RevoUtils', 'stats', 'graphics',
       'grDevices', 'utils', 'datasets', 'methods', 'base'],
      dtype='<U9')

### Model Selection

To select the best model, we followed a forward stepwise approach. We first built a linear model with only the fixed effects. After constructing the fixed effect predictors, we expended it to a random effect model with the intercept and slopes as outcomes. We entered different combinations of mileage and year as random effects. Predictors varied randomly by car trim was determined based on their variances. Model comparisons were performed using the Akaike information criterion (AIC). 

In [23]:
#All t ratios with absolute values  2.8 are significant at p< 0.01. 

#Two level models (Level 1= individual; level 2= modelTrim)
#cmod1 only allows intercept vary randomly by modelTrim
%R cmod1<-lmer(price~mileage+as.factor(year)+(1|modelTrim), data = usedcars)

R object with classes: ('lmerMod',) mapped to:
<RS4 - Python:0x7f381cc22cc8 / R:0x558b17e37cd0>

In [24]:
%R summary(cmod1)
# the output of R are commented below

In [25]:
# Linear mixed model fit by REML ['lmerMod']
# Formula: price ~ mileage + as.factor(year) + (1 | modelTrim)
#    Data: usedcars

# REML criterion at convergence: 1568106

# Scaled residuals: 
#     Min      1Q  Median      3Q     Max 
# -6.5647 -0.4927 -0.0215  0.4498 20.8846 

# Random effects:
#  Groups    Name        Variance Std.Dev.
#  modelTrim (Intercept) 39818367 6310    
#  Residual               5814663 2411    
# Number of obs: 85149, groups:  modelTrim, 24

# Fixed effects:
#                       Estimate Std. Error t value
# (Intercept)          1.978e+04  1.289e+03   15.35
# mileage             -7.248e-02  4.726e-04 -153.37
# as.factor(year)2015  1.018e+03  2.817e+01   36.12
# as.factor(year)2016  2.325e+03  2.972e+01   78.24
# as.factor(year)2017  2.980e+03  3.247e+01   91.80
# as.factor(year)2018  7.451e+03  5.861e+01  127.13

# Correlation of Fixed Effects:
#             (Intr) mileag a.()2015 a.()2016 a.()2017
# mileage     -0.019                                  
# as.fc()2015 -0.020  0.274                           
# as.fc()2016 -0.020  0.331  0.700                    
# as.fc()2017 -0.021  0.440  0.699    0.683           
# as.fc()2018 -0.014  0.348  0.432    0.412    0.445  
# fit warnings:
# Some predictor variables are on very different scales: consider rescaling

In [26]:
#cmod2 allows mileage vary randomly by modelTrim
%R cmod2<-lmer(price~mileage+as.factor(year)+(mileage|modelTrim), data = usedcars)

R object with classes: ('lmerMod',) mapped to:
<RS4 - Python:0x7f381d5ecbc8 / R:0x558b17223b38>

In [27]:
%R summary(cmod2)

In [28]:
# Linear mixed model fit by REML ['lmerMod']
# Formula: price ~ mileage + as.factor(year) + (mileage | modelTrim)
#    Data: usedcars

# REML criterion at convergence: 1556579

# Scaled residuals: 
#     Min      1Q  Median      3Q     Max 
# -6.4109 -0.4965 -0.0135  0.4630 22.3821 

# Random effects:
#  Groups    Name        Variance  Std.Dev.  Corr 
#  modelTrim (Intercept) 2.333e+07 4.830e+03      
#            mileage     1.667e-03 4.083e-02 -0.62
#  Residual              5.072e+06 2.252e+03      
# Number of obs: 85149, groups:  modelTrim, 24

# Fixed effects:
#                       Estimate Std. Error t value
# (Intercept)          2.001e+04  9.866e+02   20.29
# mileage             -8.149e-02  8.355e-03   -9.75
# as.factor(year)2015  1.161e+03  2.643e+01   43.91
# as.factor(year)2016  2.288e+03  2.794e+01   81.87
# as.factor(year)2017  2.845e+03  3.056e+01   93.09
# as.factor(year)2018  6.371e+03  5.647e+01  112.83

# Correlation of Fixed Effects:
#             (Intr) mileag a.()2015 a.()2016 a.()2017
# mileage     -0.624                                  
# as.fc()2015 -0.024  0.014                           
# as.fc()2016 -0.024  0.018  0.700                    
# as.fc()2017 -0.026  0.024  0.695    0.684           
# as.fc()2018 -0.016  0.019  0.407    0.397    0.442  
# fit warnings:
# Some predictor variables are on very different scales: consider rescaling
# convergence code: 0
# Model failed to converge with max|grad| = 30.1879 (tol = 0.002, component 1)
# Model is nearly unidentifiable: very large eigenvalue
#  - Rescale variables?
# Model is nearly unidentifiable: large eigenvalue ratio
#  - Rescale variables?

In [29]:
%R cmod3<-lmer(price~mileage+as.factor(year)+(mileage+as.factor(year)|modelTrim), data = usedcars)

R object with classes: ('lmerMod',) mapped to:
<RS4 - Python:0x7f381c086ec8 / R:0x558b15609208>

In [30]:
%R summary(cmod3)

In [31]:
# Linear mixed model fit by REML ['lmerMod']
# Formula: price ~ mileage + as.factor(year) + (mileage + as.factor(year) |  
#     modelTrim)
#    Data: usedcars

# REML criterion at convergence: 1547380

# Scaled residuals: 
#     Min      1Q  Median      3Q     Max 
# -6.6616 -0.4689 -0.0148  0.4342 23.4186 

# Random effects:
#  Groups    Name                Variance  Std.Dev.  Corr                   
#  modelTrim (Intercept)         1.339e+07 3.659e+03                        
#            mileage             5.218e-04 2.284e-02 -0.60                  
#            as.factor(year)2015 1.171e+06 1.082e+03  0.53 -0.25            
#            as.factor(year)2016 8.395e+06 2.897e+03  0.24 -0.03  0.24      
#            as.factor(year)2017 4.379e+06 2.093e+03  0.32 -0.21  0.34  0.48
#            as.factor(year)2018 1.178e+07 3.433e+03  0.30 -0.18  0.29 -0.02
#  Residual                      4.530e+06 2.128e+03                        
           
#   0.46
      
# Number of obs: 85149, groups:  modelTrim, 24

# Fixed effects:
#                       Estimate Std. Error t value
# (Intercept)          1.938e+04  7.480e+02  25.914
# mileage             -7.042e-02  4.703e-03 -14.975
# as.factor(year)2015  1.383e+03  2.234e+02   6.190
# as.factor(year)2016  2.539e+03  5.927e+02   4.284
# as.factor(year)2017  3.809e+03  4.295e+02   8.867
# as.factor(year)2018  6.366e+03  7.111e+02   8.951

# Correlation of Fixed Effects:
#             (Intr) mileag a.()2015 a.()2016 a.()2017
# mileage     -0.597                                  
# as.fc()2015  0.516 -0.241                           
# as.fc()2016  0.233 -0.029  0.243                    
# as.fc()2017  0.317 -0.204  0.344    0.485           
# as.fc()2018  0.296 -0.168  0.289   -0.019    0.458  
# fit warnings:
# Some predictor variables are on very different scales: consider rescaling
# convergence code: 0
# unable to evaluate scaled gradient
# Model failed to converge: degenerate  Hessian with 4 negative eigenvalues

In [32]:
%R cmod4<-lmer(price~mileage+as.factor(year)+(as.factor(year)|modelTrim), data = usedcars)

R object with classes: ('lmerMod',) mapped to:
<RS4 - Python:0x7f381c1047c8 / R:0x558b1969a4b8>

In [33]:
%R summary(cmod4)

In [34]:
# Linear mixed model fit by REML ['lmerMod']
# Formula: price ~ mileage + as.factor(year) + (as.factor(year) | modelTrim)
#    Data: usedcars

# REML criterion at convergence: 1550905

# Scaled residuals: 
#     Min      1Q  Median      3Q     Max 
# -6.3907 -0.4656 -0.0176  0.4326 22.8028 

# Random effects:
#  Groups    Name                Variance Std.Dev. Corr               
#  modelTrim (Intercept)         24703512 4970                        
#            as.factor(year)2015  2142318 1464     0.81               
#            as.factor(year)2016  3747871 1936     0.76 0.83          
#            as.factor(year)2017  8352670 2890     0.62 0.77 0.90     
#            as.factor(year)2018 15229435 3902     0.59 0.73 0.82 0.94
#  Residual                       4731733 2175                        
# Number of obs: 85149, groups:  modelTrim, 24

# Fixed effects:
#                       Estimate Std. Error t value
# (Intercept)          1.925e+04  1.015e+03   18.96
# mileage             -6.798e-02  4.362e-04 -155.85
# as.factor(year)2015  1.418e+03  3.007e+02    4.72
# as.factor(year)2016  2.628e+03  3.969e+02    6.62
# as.factor(year)2017  3.928e+03  5.915e+02    6.64
# as.factor(year)2018  6.490e+03  8.043e+02    8.07

# Correlation of Fixed Effects:
#             (Intr) mileag a.()2015 a.()2016 a.()2017
# mileage     -0.023                                  
# as.fc()2015  0.805  0.023                           
# as.fc()2016  0.755  0.024  0.832                    
# as.fc()2017  0.617  0.025  0.767    0.901           
# as.fc()2018  0.581  0.026  0.726    0.813    0.931  
# fit warnings:
# Some predictor variables are on very different scales: consider rescaling

In [35]:
%R AIC(cmod1, cmod2, cmod3, cmod4)
#adopt cmod4

,df,AIC
cmod1,8.0,1.568312e+06
cmod2,10.0,1.556819e+06
cmod3,28.0,1.547685e+06
cmod4,22.0,1.551155e+06


The final model was a linear mixed model including mileage and the categorical year of manufacture variables as fixed effects; and only the year variable as the random effect. In this model, only year (variance= 2,142,318, 3,747,871, 8,352,670, 15,229,435; year 2015 to 2018, respectively) varied randomly by car trim. Although the model including both mileage and year as random effects (AIC= 1,547,436) had lower AIC than the final model (AIC= 1,550,949), that model was not selected because of the small variance of mileage across levels (variance= .0005).

The following system of equations shows a basic prediction line as a linear mixed model:

<img src="equation.png.png">

Where dependent variable Y is the price of used car (in dollars); M is a numeric variable stands for the mileage of used car (in miles); Yr2015 to Yr2018 are the years of manufacture (ex. for Yr2015, Yr2015=0 if the car was manufactured in 2014; Yr2015=1 if the car was manufactured in 2015). ${ε}_{ij}$ stands for the error term that cannot be explained by the equation. For each coefficient ${β}_{i j}$,  ${γ}_{i 0}$  stands for the fixed term and  ${μ}_{i j}$ indicates the random terms (i= 0,1,2,…; j=0,1,2,…).

### Interpretation

#### Fixed effect
The final model is presented in Table X1. The linear mixed model suggests that, ignoring the random effects, every 1,000-mile increase in mileage decreases the car value by 67.98 (p<.001). Compared to a car manufactured in 2014, that manufactured 2015 is 1,418 more expensive (p<.001); that manufactured 2016 is 2,628 more expensive(p<.001); that manufactured 2017 is 3,928 more expensive(p<.001); and that manufactured 2018 is 6,490 more expensive (p<.001).

<img src="tablex1.png">

#### Random Effect
According to the linear mixed model, year of manufacture varied randomly by car trim. The predicted value for a car manufactured in 2014 ranged from 9,681.50 (Ford Focus S) to 30,272.15 (Ford F-150 Lariat), while for that manufactured in 2018, the price ranged from 15,560.65 (Hyundai Elantra SE) to 44,704.63 (Ford F-150 Lariat). Overall, Ford F-150 Lariat manufactured from 2014 to 2018 had the highest predicted value compared to other cars manufactured in the same year. In terms of the lowest predicted value, Ford Focus S manufactured in 2014, 2015 and 2016, and Hyundai Elantra SE manufactured in 2017 and 2018 had the lowest predicted price compared to other cars manufactured in the same year. (Table X2). 

<img src="tablex2.png">

To better illustrate our results, the effects of mileage and year of manufacture combined on the car price were depicted in a graph (Figure X). Mileage was controlled at the annual mean level for each car trim. 

In [36]:
%R usedcars$predictPrice<-predict(cmod4)
#Too many numbers, unable to graph...

#Calculate predicted values for each model trim with mileage controlled at the mean
%R mtmean<-aggregate(mileage~ year+modelTrim, data = usedcars, FUN = mean)
%R mtmean$predictPrice<-predict(cmod4, mtmean)
a = %R mtmean
pd.set_option('display.max_rows', 200)
a

,year,modelTrim,mileage,predictPrice
1,2014,Audi A4 2.0T Premium quattro,44506.814371,19505.885311
2,2015,Audi A4 2.0T Premium quattro,37522.106977,21746.649877
3,2016,Audi A4 2.0T Premium quattro,37753.262136,24480.282265
4,2017,Audi A4 2.0T Premium quattro,14845.420382,31236.164145
5,2018,Audi A4 2.0T Premium quattro,4662.352941,38005.686767
6,2014,Chevrolet Malibu 1LT,56145.749326,13026.098093
7,2015,Chevrolet Malibu 1LT,35710.306039,14782.363592
8,2016,Chevrolet Malibu 1LT,32163.115950,16657.347653
9,2017,Chevrolet Malibu 1LT,31667.275430,16942.175843
10,2018,Chevrolet Malibu 1LT,15410.404762,20230.139498


Plot the line chart of the dataframe above for data visualization.

In [37]:
%R library(ggplot2)
p1 <- ggplot(mtmean, aes(x = year, y = predictPrice, color = modelTrim, linetype= modelTrim))+
  geom_line(aes(y = predictPrice), size= .3)+
    scale_colour_manual(values= c(rep(c('#000000', '#999999', '#ff0000', '#fff200', '#00ff11', '#00fffa'),4)))+
  scale_linetype_manual(values= c(rep(c("solid", "dashed", "dotted", "dotdash", "longdash", "twodash", "1F", "F1"),3)))+
  xlab('Year')+ ylab('Predicted Price (in thousand dollars)')+
  scale_x_reverse()+
  scale_y_continuous(limits = c(0, 50000), labels = seq(0,50,10))+
  theme(panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(),
        panel.background = element_blank(),
        axis.line = element_line(colour = "black"),
        legend.position="bottom",
        legend.title=element_blank(),
        axis.text = element_text(size = 11))

<img src="24.png">

Next, we calculate the price drop ratio of each modelTrim.

In [38]:
name_list = a['modelTrim']
name_list = list(sorted(set(name_list)))

In [39]:
from collections import OrderedDict
ans = []
for i in range(24):
    Price_diff = a.iloc[i*5+3]['predictPrice'] - a.iloc[i*5+1]['predictPrice']
    ans.append(OrderedDict({'modelTrim':name_list[i], 'purchase price': a.iloc[i*5+3]['predictPrice'], 'price drop after 2 years':Price_diff, 'price drop ratio':Price_diff/ a.iloc[i*5+1]['predictPrice']}))

In [40]:
compare = pd.DataFrame(ans)

In [41]:
compare.sort_values(by=['price drop ratio'])

,modelTrim,purchase price,price drop after 2 years,price drop ratio
12,Hyundai Elantra SE,13015.331422,509.543129,0.040745
4,Ford F-150 XLT,31321.558189,1574.016886,0.052913
21,Toyota Camry SE,17448.300961,1136.103846,0.069648
20,Toyota Camry LE,17208.635051,1672.752559,0.107670
3,Ford F-150 XL,30366.619885,3084.593943,0.113063
7,Ford Focus Titanium,16760.656574,2021.004967,0.137113
6,Ford Focus SE,13794.671049,1700.221569,0.140579
1,Chevrolet Malibu 1LT,16942.175843,2159.812251,0.146107
2,Ford F-150 Lariat,41785.417239,5333.616235,0.146320
22,Toyota Camry XLE,21213.651203,2734.128738,0.147955


### Result of lab2:

Hyundai Elantra SE still dominated this ranking. 

We can also see that the ranking of Ford F-150 dropped significantly due to its high price tag.

German brands are at the bottom of our rankings.

Next, we evaluate the mean price dropped in two years for all car models across 24 modelTrim. To do so, we first need to get the mean mileage for each year and use that as out input to our pricing function.

In [42]:
mileage_mean_list = database.groupby(['year'])['mileage'].mean()
mileage_mean = (mileage_mean_list.iloc[0] - mileage_mean_list.iloc[4])/4
mileage_mean

13116.514606088316

We set the year of car as 2017, the mileage as 10000 miles, the ModelTrim as Hyundai Elantra SE.

In [43]:
%R Year_now <- 2017
%R Mileage <- 10000
%R ModelTrim <- "Hyundai Elantra SE"

array(['Hyundai Elantra SE'],
      dtype='<U18')

In [44]:
%R Year <- 2015
%R Milage_nor <- Mileage + 13116.514606088316 * 2

%R mtdf<-data.frame(seq(2014,2018,1), Milage_nor, ModelTrim)
%R colnames(mtdf)<-c("year","mileage","modelTrim")
%R mtdf$pred<-predict(cmod4, mtdf,re.form=~(as.factor(year)|modelTrim))
Price_nor = %R mtdf$pred[mtdf$year==Year]
print('Price_predicted: ',Price_nor[0])

Price_predicted:  12455.8876067


The price of this car after 2 years is predicted as 12455.87796 dollars. This vaule can be used for guide.

## Future Work

1. Use LSTM to analyze pricing information 
LSTM has shown to be quite good when used to model and predict time series data. We can use it to model our pricing function using LSTM and compare the results to ours.

2. Use Logarithmic Regression
Pricing for cars does not necessarily follow a Linear pattern. The pricing curve smooths out after a few years after warranty expires. Using a Logarithmic Regression makes sense and could be more useful to us when calculating the prices of cars.

## Summary and References

Given the results of this project, we can see that used cars within a year are a lot cheaper than brand new cars and offers very good value for the price. However, new cars do provide a much broader range of options for you to choose from. 

For used car markets, Hyundai Elantra is the best car that retain its value after two years with only a 4% deduction in prices. Japanese and Korean cars can generally retain more percentages of its value after two years compare to US and Germany manufacturers, with the exception of Ford F-150 which performed fantastically.

1. https://cars.com
2. http://www.bodowinter.com/tutorial/bw_LME_tutorial2.pdf
3. https://stats.idre.ucla.edu/other/mult-pkg/introduction-to-linear-mixed-models/
4. https://www.kbb.com/
5. https://cran.r-project.org/web/packages/lme4/index.html